# load thư viện

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install SentencePiece
!pip install transformers
!pip install transformers[torch]
!pip install pypi
!pip install accelerate -U
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

# hàm tiền xử lý dữ liệu

In [3]:
!pip install py_vncorenlp
!mkdir -p vncorenlp
import py_vncorenlp
import re
py_vncorenlp.download_model(save_dir='/content/vncorenlp')
vncore = py_vncorenlp.VnCoreNLP(save_dir='/content/vncorenlp')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.5 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=8390fedb423d7290186f396bd4c6a17fa8b7d07d093d20ba716ff725f38bd2d2
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [4]:
# xóa đi các dấu cách thừa, các từ không cần thiết
def normalize_text(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
    text = re.sub(r'\s+',' ',text).strip()
    return text


# tách từ bằng vncorenlp
def tokenize_text(text):
    return vncore.word_segment(text)

# lọc bỏ stopwords
stopwords = set(open('/content/drive/MyDrive/IE403/Dataset/Syn-VSFC/vietnamese-stopwords-dash.txt').read().splitlines())
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords]

# đưa về chữ thường
def to_lowercase(tokens):
    return [token.lower() for token in tokens]

# hàm tiền xử lý văn bản
def preprocess_text(text):
    text = normalize_text(text)
    tokens = tokenize_text(text)
    tokens = remove_stopwords(tokens)
    tokens = to_lowercase(tokens)
    return ' '.join(tokens)

# load mô hình

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from transformers import TrainingArguments, Trainer
import torch

In [36]:
model_path = "/content/drive/MyDrive/IE403/Model/phobert/syn_phobert.pth"

In [37]:
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [10]:
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [38]:
def predict_sentiment(sentence):
    """
    Predicts the sentiment of a sentence using a pre-trained model.

    Args:
        sentence: The sentence to be analyzed.

    Returns:
        A tuple containing the predicted sentiment label and the prediction probabilities.
    """

    # Preprocess the sentence
    input_sentence = preprocess_text(sentence)

    # Load the model and tokenizer
    # model_path = "/content/drive/MyDrive/IE403/Model/phobert/syn_phobert.pth"
    # model = AutoModelForSequenceClassification.from_pretrained(model_path)
    # tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')

    # Tokenize the sentence and convert it to a tensor
    inputs = tokenizer(input_sentence, return_tensors="pt")

    # Get the model predictions
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

    # Convert the prediction to a label
    labels = ['Tiêu cực', 'Trung lập', 'Tích cực']
    predicted_label = labels[predictions.item()]

    # Get the prediction probabilities
    probabilities = torch.softmax(outputs.logits, dim=1)

    return predicted_label, probabilities

In [52]:
# Example usage
sentence = ""
predicted_label, probabilities = predict_sentiment(sentence)

print(f"Predicted label: {predicted_label}")
print(f"Prediction probabilities: {probabilities}")


Predicted label: Trung lập
Prediction probabilities: tensor([[0.0044, 0.9085, 0.0871]], grad_fn=<SoftmaxBackward0>)


In [55]:
!pip install joblib
import joblib

In [56]:
svm = joblib.load('/content/drive/MyDrive/IE403/Model/Syn_svm_model.joblib')

In [57]:
sentence_pred = svm.predict()

ValueError: could not convert string to float: 'dạy dễ hiểu , vui tính .'